Este archivo contiene la implementación - Funciones, clases y sus definiciones - del simulador, hecho para la beca de investigación y la tesis

# Importamos librerias

In [33]:
import numpy as np
from numpy import pi, sin, cos, abs, log2
from scipy.fft import fft, ifft
from scipy.io import wavfile
from scipy.signal import lfilter
from scipy import signal
import librosa, librosa.display
import matplotlib.pyplot as plt

from IPython.display import Audio
#from ipywidgets import interactive, interact
#from matplotlib import rc
#import matplotlib.animation as animation
#rc('animation', html='jshtml')

import os
#os.chdir(r'C:\Users\Graciela\Desktop\Audio_Code\carriers')

# Implementación y Parámetros:

Nota: Hay que recargar los electrodos al modificiar algunos parámetros

In [ ]:
#Parámetros y Constantes
IMPLANT_SR = 16000 #se setea abajo, en el implante
OUT_SR = 96000

#MAXIMA (para estrategia ACE)
MAXIMA = 8 #Dejar como 0 para usar todos los canales disponibles

#MISMATCH: Corrimiento extra en la cóclea (hacia la base medido en mm). Valores positivos es que le faltó profundidad, negativos es que fue profundo de más
MISMATCH = 0

#COCHLEAR DUCT LENGHT (mm): Longitud del organo de corti (usado al momento de calcular las freqs de los electrodos tipo Lateral wall con la funcion de Greenwood)
CDL_OC = 35   # un criterio es 33.13 +- 2.11 mm (olga)  35.58 ± 1.41 mm (kawano)
CDL_MW = 20   # un criterio es  15.49 +- 0.69     18.29 ± 1..47 mm (kawano). Como el contour advance es un poco más largo porque no es que arranca en mw sino que tiene una curvita le ajusto acá haciendo más larga
#COCHLEAR DUCT LENGHT (mm): Longitud de pared modiolar (usado al momento de calcular las freqs de los electrodos tipo Modiolar wall con la funcion de Greenwood)


#DISPERTION = Parámetro que uso para simular el "broadness" de la región del excitación de los electrodos.
#0 = Filtro angosto
#1 = Filtro de electrodo a electrodo (media geométrica)
#2 = Filtros cada 2 electrodos aprox
#Toma float >= 0
DISPERTION = 1
D = 1 #lo mismo pero representa el ancho de las bandas criticas en mm de los electrodos

#Orden de los filtros de análisis y resíntesis
ORDEN = 6

#Si le voy a hacer escuchar la salida a un implantado o no
EXP_OYENTE = True

#Extracción de envolvente: Método Rectify+LP (Cochlear)
ENVELOPE_FC = 200
ENVELOPE_FULLWAVE = True # En el implante, se extrae solo media onda para simular el disparo de las células
ENVELOPE_ZERO_PHASE = True #Especifica si se

FILTERS_ZERO_PHASE = False

#MODO DE SINTESIS
INTERLEAVED = 1 #Es ACE, pero con estimulación interleaved
ACE = 2       #Es ACE, pero estimulando todo en paralelo al mismo tiempo
CONTINOUS  = 3 #Es oomo modo ACE, pero usando todos los canales disponibles
MODE = ACE

#ESTIMULACION
PW  = 0 #de cada electrodo, en segundos . Debe ser menor a 1/(n*pps). Dejar como 0 para que se asigne al máximo posible50*(10**-6).
PPS = 0 #al menos 4 veces la frec de corte del filtro de envolvete si se usa en un caso real.
WIN_LEN_ACE = 4 #En ms. Tipicamente 4 o 8En ms. Si se activa auto, se hace por ciclo de estimulación. Creo que Medel 4 y Cochlear 8.
WIN_LEN_AUTO = False #Desactive el debugeo al setear los parametros del array

#Usar esta funcion para setear el pps y pw, que indica los límites de "seguridad" y da info
def SETEAR_PPS(pps, pw = 0):
  global PPS, PW
  #Lo correcto es setear el pps primero y dejar pw como un parámetro con restricción. Para este caso, elijo un pw demasiado grande, irreal, pero que dé un efecto audible
  #En este caso elijo que pw abarque todo el tiempo de 1/n.pps
  PPS = pps
  PW = pw
  if(MODE == INTERLEAVED and 1/(MAXIMA*PPS) < PW):
    print('ERROR: Ancho de pulso muy grande para la frecuencia especificada.')
    print('El ancho de pulso máximo para la frecuencia de ' + str(PPS) + ' pps es de ' + str(round(1000000/MAXIMA/PPS,2)) +'us' )
  if(pw == 0): PW = 1/(MAXIMA*PPS)
  print('Samples por pulso: ' + str(round(PW*OUT_SR)))
  print('Pulse width: ' + str(round(PW*1000000)) + 'us')

SETEAR_PPS(800)


#CROSSTALK
CHANNEL_INTERACTION_INDEX = 1 #Qué tanta interacción incluyo. 0 = Nada. Es un "control de ganancia" del efecto
                              #A partir de la versión 4, en "1" toma valor "normalizado" según estudios (patente CSSS de Medel).
                              #Tambien a partir de v4 hay que recalcular los coeficientes de interacción al cambiar este parámetro (llamar a "array.setearParametros()")
                              #viejo
ASYMMETRY_INDEX = 0 #qué tanto más se filtra hacia los electrodos basales que apicales.
                      # 0 = se filtra en ambas direcciones por igual.
                      # 1 = se filtra solo en direccion basal.
                      #Retomado en la v4

#ANALISIS: EL ULTIMO CANAL ES FILTRO PASAALTOS O ES BP
LASTCHANNELHP = False

#COMPRESION DEL RANGO DINÁMICO (valores globales)
T_SPL = 25 # (from 9 to 50 dB SPL) . 25 = Default
C_SPL = 65 # (from 65 to 84 dB SPL)  65 = Default
IIDR = C_SPL - T_SPL
VOLUME_LEVEL = 1
SENSITIVITY = 12 # ranges from a minimum of “1” to a maximum of “20. The default sensitivity setting is “12,” which results in the IIDR being positioned from 25 dB SPL (T-SPL) to 65 dB SPL (C-SPL).


#FABRICANTES
COCHLEAR = 'Cochlear'
MEDEL = 'Med-El'
AB = 'Advanced Bionics'


#TIPOS DE ARRAYS
MW = 'Modiolar Wall'
LW = 'Lateral Wall'

DEBUG = True

Samples por pulso: 15
Pulse width: 156us


#Funciones importantes

Todo: Reformular todo en una clase

In [ ]:
def envelope_Cochlear(s, fc, fullwave, gain_compensation = False): # Recity + LP
  envelope = np.zeros(len(s))
  if(fullwave):
    for i in range(len(s)): envelope[i] = abs(s[i])
    envelope *= np.pi/2 #factor de correccion cuando la entrada es una sinusoide. Sale de aplicar |s(t)| y ver su serie de Fourier.

  else: #half_wave
    for i in range(len(s)): envelope[i] = max(0., s[i])

  #Low - Pass: Butter de 2do orden
  if(ENVELOPE_ZERO_PHASE):
    b,a = signal.butter(1, fc, 'lp', output='ba', fs = IMPLANT_SR) #Orden 1 porque se aplica dos veces
    envelope =  signal.filtfilt(b,a, envelope)
    #sos = signal.butter(2, 200, 'lowpass', fs=IMPLANT_SR, output='sos')
    #envelope = signal.sosfiltfilt(sos, envelope)

  else:
    sos = signal.butter(2, fc, 'lowpass', fs=IMPLANT_SR, output='sos')
    envelope = signal.sosfilt(sos, envelope)

  if (gain_compensation): envelope *= max(s)/max(envelope) # 1.6 aprox, malo para transientes (pero en esta etapa se supone que ya no hay)
  return envelope

def envelope_Hilbert(x):
  n = len(x)
  Z = np.zeros(n, dtype='complex')
  X = fft(x)
  for i in range(1,n//2): Z[i] = 2.*X[i]
  Z[n//2] = X[n//2]
  z = ifft(Z)
  env = np.abs(z)
  return env

#Genera una señal de ruido blanco de n samples
def wnoise_sig(n):
   return 2*(np.random.rand(n) - .5) #ruido del mismo largo de la señal

#Genera una señal de ruido blanco PLANA EN EL ESPECTRO de n samples
def wnoise_sig2(n):
    phases = np.random.rand(n)*2*np.pi
    return ifft(np.cos(phases) + 1j*np.sin(phases))

#Genera un tono puro de frecuencia f, duracion en segundos
def puretone(f, dur, sr, phase = 0):
  return np.sin([ 2*pi*f*i/sr for i in range(round(sr*dur))])

#Genera un tono puro de frecuencia f, duracion en segundos, con el vibrato depth especificado (1 = un semitono hacia arriba y hacia abajo, 2 = un tono, etc)
def puretoneVib(f0, dur, sr, vibDepth, vibFreq):
  N = round(sr*dur)
  out = np.zeros(N)

  # Quiero que la frecuencia instantanea f sea  = 2**( vibDepth/12*sin(2*pi*vibFreq*ti))
  #Esto es la derivada de la fase. Tengo que integrar esa función para obtener la fase instantánea
  #Uso una aproximación lineal,  phi(t+dt) \approx phi(t) + f(t)*dt
  # Y phi(0) = 0
  phi = 0
  dt = 1/sr
  ti = 0

  for i in range(N):
    out[i] = sin(2*pi*phi)
    dphi = f0 * 2**(vibDepth/12 * sin(2*pi*vibFreq*ti))
    phi += dt * dphi
    ti += dt
    
  return out

  
#Genera un tono puro de frecuencia f, duracion en segundos, con el vibrato depth especificado (1 = un semitono hacia arriba y hacia abajo, 2 = un tono, etc)
#att es el tiempo en segundos (no milis) que tarda en entrar 100% el vibrato. Por defecto, hago hold la mitad del tiempo y luego una rampita
def puretoneVib2(f0, dur, sr, vibDepth, vibFreq, vibAtt = 0.01):
  N = round(sr*dur)
  out = np.zeros(N)

  # Quiero que la frecuencia instantanea f sea  = 2**( vibDepth/12*sin(2*pi*vibFreq*ti))
  #Esto es la derivada de la fase. Tengo que integrar esa función para obtener la fase instantánea
  #Uso una aproximación lineal,  phi(t+dt) \approx phi(t) + f(t)*dt
  # Y phi(0) = 0
  phi = 0
  dt = 1/sr

  #Ramp con hold
  vibRamp = []
  for i in range(N//2): vibRamp.append(0) #hold
  for i in range(N//2 + 1): vibRamp.append( min( 2*i/sr / vibAtt, 1)) #start ramp 
  #for i in range(N): vibRamp.append( min(1000*i/sr / vibAtt, 1)) Ramp de una, sin hold

  for i in range(N):
    try: out[i] = sin(2*pi*phi)
    except: Exception('HOLA')
    dphi = f0 * 2**(vibDepth/12 * sin(2*pi*vibFreq*i*dt) *vibRamp[i] )
    phi += dt * dphi
  return out

#Genera un tono puro de frecuencia f, duracion en segundos
#Devuelve la fase actualizada a la que llegó
def puretonePhase(f, dur, sr, phase = 0):
  N = round(sr*dur)
  out = [] 
  dp = 2*pi*f/sr
  for _ in range(N):
    out.append(sin(phase))
    phase += dp

  return out, phase
  
#Genera una nota de amplitud fija de frecuencia f y amplitud de cada armonico dada en la lista, duracion en segundos
def pureNote(f, amps, dur, sr):
  if(f==0): return np.zeros(dur*sr)
  amps_sinalias = [amps[i] for i in range(len(amps)) if (i+1)*f < sr/2]
  return 1/len(amps_sinalias)* np.sum([puretone(f*(i+1), dur, sr)*amps_sinalias[i] for i in range(len(amps_sinalias))] , axis=0)

#Genera una señal melódica
#fs son las frecuencias de las notas. 0 = Silencio
#amps = amplitud de cada parcial. Fijo para todas las notas
#durs: duracion en segundos de cada nota
#sr: sampling rate
#(no tienen envolvente espectral)
def melody(fs, amps, durs, sr):
  out = []
  for i in range(len(fs)):
    out += list(pureNote(fs[i], amps, durs[i], sr))
  return np.array(out)

#recibe midi notenumber, devuelve Hz temperados
def midi2freq(notenumber):
  return 440.* 2**((notenumber-69)/12)

#recibe dos midi notenumbers, devuelve intervalo (ratio de frecuencias)
#todo: Para ir a los graves, los ratios son otros
def midi2ratio(nn1, nn2):
  i = 0
  dn = abs(nn2-nn1) % 12
  if(nn1 == 0 or nn2 ==0): return 0
  if(dn ==0): i = 1
  #Escala Mayor
  if(dn ==2): i = 9/8 #o 8/7 o 10/9
  if(dn ==4): i = 5/4
  if(dn ==5): i = 4/3
  if(dn ==7): i = 3/2
  if(dn ==9): i = 5/3
  if(dn ==11):i= 15/8
  #if(dn==12): i = 2
  #Alteraciones
  if(dn ==1):i = 16/15 # o 7/6
  if(dn ==3):i = 6/5 # o 7/6
  if(dn ==6):i = 7/5
  if(dn ==8):i = 8/5
  if(dn==10):i = 7/4

  #Octava
  i*=  2 ** (abs(nn2-nn1)//12 )
  #Intervalo descendente
  if(nn1 > nn2): i = 1/i
  return i

#ruido filtrado por pasabanda
def filtered_noise(fc, df, order, dur, sr):
  w = wnoise_sig(round(dur*sr))
  sos = signal.butter(order//2, [fc-df/2, fc+df/2], 'bandpass', fs=sr, output='sos')
  return signal.sosfilt(sos, w)


def adsre(a, d, s, r, x, sr): #adr en ms. x es la señal a aplicarle la envolvente. devuelve una copia con la envolvente aplicada
                             
  out = x.copy()
  out = np.array(out)

  Na = round(a*sr/1000)
  Na = min(Na, len(x)) #seguridad, por las dudas... que la secuencia dure menos que el tiempo de ataque
  att = np.exp([ 7*(i/Na-1) for i in range(Na)])
  out[0:Na] *= att

  Nd = round(d*sr/1000)
  Nd = min(Nd, len(x[Na:])) #seguridad, por las dudas...
  dec = np.exp([ np.log(s)*(i/Nd) for i in range(Nd)]) #s^(i/N)
  out[Na:Na+Nd] *= dec

  Nr = round(r*sr/1000)
  Nr = min(Nr, len(x[-Nr:]), Na + Nd) #seguridad, por las dudas...
  rel = s*np.exp([ 7*(-i/Nr) for i in range(Nr)])

  if(Nr > 0):
    out[-Nr:] *= rel
  
  if(Nr>0):
    out[Na+Nd: -Nr] *= s

  return out

#Tukey window
def Tukey(N, a):
  out = np.ones(N)
  aN = a*N
  for i in range(round(aN/2)):
    out[i] = .5 - .5*cos(2*pi*i/aN)
  for i in range(N//2):
    out[N-i-1] = out[i]
  return out

'''
Toma una señal y un array de electrodos, y le aplica filtros pasabanda en paralelo
Si "analisis==True" el centro de los filtros es de los de análisis del procesador
Si no, entra en modo resíntesis y el centro de los filtros es el de las frecuencias características estimadas de cada electrodo
Devuelve un array con las señales filtradas
'''
def filter_bank_CIS(s, elec_array, orden = ORDEN, analisis = True):

  #Todo: Que el primer low cut sea mucho más pronunciado (más orden)
  if(analisis):
    return filter_bank_analisis(s,elec_array, orden)

  else: #resíntesis
    return filter_bank_sintesis(s,elec_array, orden)

def filter_bank_analisis(s, elec_array, orden):
  N = elec_array.Nchann
  filtered_bank = [] #Out (array con todas las señales filtradas)

  #frecuencias centrales de los filtros:
  freqs = elec_array.freqsAnalisis #Etapa de analizar la señal entrante

  #Frecuencia de corte para el canal más grave
  if elec_array.channel1_lowcut != None: fizq = elec_array.channel1_lowcut
  else: fizq = 150 # freq izq de la primer banda, la asumo
  #fs = IMPLANT_SR (dejar? ver si hace falta que sea parametro o si puedo dejar acá)

  #Si el array ya me da las frecuencias exactas de las bandas de paso (en vez de las freqs centrales), voy con esas
  if(type(freqs[0]) in [type([0,1]), type(np.array([0,1]))] ):
    for i in range(N):
      f1 = freqs[i][0]
      f2 = freqs[i][1]
      filtered_bank.append(aplicar_filtro(s, orden, IMPLANT_SR, f1, f2))

  #sino, las calculo: (Calculo de frecucencias de corte)
  if(type(freqs[0]) in [type(123), type(123.456)]) :
    for i in range(N-1):   #e1 --> N-1  el ultimo lo hago highpass

      fc = freqs[i]
      fder = freqs[i+1]

      '''
      MODO LINEAL:
        Si las frecuencias centrales son 100, 200, 300, filtro del medio va a tener banda de paso [150,250]
        O sea, hay un overlaping mucho mayor entre bandas haciendolo de esta manera que de la otra
      '''
      # Posible Q = 3 =  fc/BW  (fuente: Patente de CSSS). Pero da anchos muy angostos
      # TODO: Hacer pruebas con esto.
      if(elec_array.manufacturer == MEDEL):
        Q = 3  # (f1+ f2)/ 2(f2-f1)
        f2 = fc*(1+1/(2*Q))
        f1 = fc*(1-1/(2*Q))

      f1 = (fizq + fc)/2
      f2 = (fc + fder)/2
      fizq = fc
     #TODO: A los de Medel aumentarles el overlap

      filtered_bank.append(aplicar_filtro(s, orden, elec_array.SR, f1, f2))

    #Ultimo canal: Pasaaltos
    if(LASTCHANNELHP):
      sos = signal.butter(orden, freqs[-1], 'highpass', fs=elec_array.SR, output='sos')
      filtered_bank.append(signal.sosfilt(sos, s))
    else: #Sigo bp (tecnicamente es lo mismo pero la freq de corte la corri un poquito pa atras digamos )
      
      fc = freqs[-1]
      fder = elec_array.SR / 2 - 1
      f1 = (fizq + fc)/2
      filtered_bank.append(aplicar_filtro(s, orden, elec_array.SR, f1, fder))


  return filtered_bank

def filter_bank_sintesis(s, elec_array, orden):
  N = elec_array.Nchann
  filtered_bank = [] #Out (array con todas las señales filtradas)
  #print('DISPERTION: ', DISPERTION)

  if(EXP_OYENTE):     freqs = elec_array.freqsElectrodos #Etapa de simular la estimulación de los electrodos ("resintesis")
  if(not EXP_OYENTE): 
    freqs = elec_array.freqsAnalisis  #Le voy a hacer escuchar a un implantado... le sintetizo las mismas freqs que para el analisis 
    if(type(freqs[0]) in [type([0,1]), type(np.array([0,1]))] ):
      for i in range(len(freqs)): freqs[i] = freqs[i][0]/2+ freqs[i][1]/2

  fizq = freqs[0]* freqs[0]/freqs[1]
  # fs = OUT_SR

  for i in range(N):
    fc = freqs[i]

    if(i<N-1): fder = freqs[i+1]
    else: fder = fc*fc/fizq
    '''
    MODO INTERVÁLICO:
      Así, el ancho de banda de los filtlros se calcula en base a los intervalos y no a la distancia entre frecuecienias
      Ej: Si las frecuencias centrales son 100 200 300,   el filtro del medio va a tener banda de paso [141, 245],  ya que el intervalo 141/100 = 200/141
      A mi entender sería mejor así, pero no es lo que se usa
    '''
    c = DISPERTION/2
    eps = 0.5 #Por si DISPERTION == 0
    f1 = fc**(1-c) * fizq**c - eps
    f2 = fc**(1-c) * fder**c + eps
    fizq = fc
    #Todo: Afectar tambien al orden?
    #Todo: Evaluar si no conviene el otro método (f1 = 2^-c .fc , f2 = 2^c.fc)

    #print('Aplicando filtro pasabanda con freqs ', round(f1), round(f2))

    orden += round(4 - DISPERTION/4) #Dispertion = 0: orden +=4.   Disperion = 0.25: orden +=3 . Disperion = 0.5: orden += 2

    filtered_bank.append(aplicar_filtro(s, orden, OUT_SR, f1, f2))
  return filtered_bank


#En esta version d es un parámetro espacial y uso greenwood 
#elecs_activos es una lista con los indices de los elecs prendidos. Ej [0,3,7,....]
#Todo Esto deberia ser metodo de clase
def filter_bank_sintesis2(s, elec_array, indices_elecs_activos = None, sr = OUT_SR):
  N = elec_array.Nchann
  filtered_bank = [] #Out (array con todas las señales filtradas)

  if(DEBUG) : print(f'Filter bank sintesis 2 \n')
  if(indices_elecs_activos is None): indices_elecs_activos = [i for i in range(N)]

  for i in indices_elecs_activos:
    #calculo posiciones de los electrodos
    x_ei = elec_array.xi - elec_array.dx*i - MISMATCH #posicion del elec
    x_apical =  x_ei + D
    x_basal = x_ei - (1-ASYMMETRY_INDEX)*D
    
    #Les asigno frecuencia con greenwood segun lateral o modiolar
    if(elec_array.type == LW):
      f1, f2 = Greenwood_p( x_apical / CDL_OC), Greenwood_p(x_basal / CDL_OC )

    if(elec_array.type == MW):

      f1, f2 = Greenwood_Perimodiolar_MW_p( x_apical / CDL_OC), Greenwood_Perimodiolar_MW_p(x_basal / CDL_OC )

    #orden de los filtros
    orden_apical = max(round(1/np.sqrt(D)),1)
    orden_basal = max(round(1/np.sqrt(1-ASYMMETRY_INDEX)/np.sqrt(D)),1)

    if(DEBUG):  print('Aplicando filtro pasabajos y altos con freqs y orden ', round(f2), round(f1), orden_apical, orden_basal)

    #LP
    sos = signal.butter(orden_apical, f2, 'lowpass', fs=sr, output='sos')
    out_i = signal.sosfilt(sos, s)

    #HP
    sos = signal.butter(orden_basal, f1, 'highpass', fs=sr, output='sos') 
    out_i  = signal.sosfilt(sos, out_i)


    filtered_bank.append(out_i)
  return filtered_bank


#pasabandas en realidad
def aplicar_filtro(s, orden, fs,f1,f2):
  #Aplicar filtro
  if(FILTERS_ZERO_PHASE): #El orden resulta 8
    sos = signal.butter(2, [f1,f2], 'bandpass', fs=fs, output='sos') #divido por 2 porque la implementación pide la cantidad de secciones de orden 2 para filtros BP
    return signal.sosfiltfilt(sos, s)
  else:
    sos = signal.butter(orden//2, [f1,f2], 'bandpass', fs=fs, output='sos') #divido por 2 porque la implementación pide la cantidad de secciones de orden 2 para filtros BP
    return signal.sosfilt(sos, s)

#Resamplea x de la frecuencia de sampleo fs1 a fs2
def resample(x, fs1, fs2):
  n = round(len(x)*fs2/fs1)
  return signal.resample(x, n)

def compress_log(s):
  return Y

def compress_power(s):

  return Y

#Solo clip
def compress_envelope_sample(sample):
  max = 10**(-(100-C_SPL)/20)
  min = 10**(-(100-T_SPL)/20)
  if sample < min : sample = 0
  if sample > max: sample = max
  return sample

#Solo clip
def compress_envelope_rms(envelope):
  max = 10**(-(100-C_SPL)/20)
  min = 10**(-(100-T_SPL)/20)
  if sample < min : sample = 0
  if sample > max: sample = max
  return sample

def Interleave(): #switchea/apaga señales de entre canalaes paralelos. TODO
  return 

'''CHANNEL INTERACTION '''

#TODO: OJO NO ANDA ESTA FUNCION!!f
def channel_interaction_CONTINOUS(filtered_noises, elec_array):
  CII = CHANNEL_INTERACTION_INDEX
  if(CII == 0): return filtered_noises
  interacted_channels = filtered_noises.copy()
  N = elec_array.Nchann

  #ei = to,  e = from.   g[from][to]
  for e in range(N): 
    for ei in range(N): #todo: en realidad, debería tomar el valor de la envolvente en el momento actual o en el momento anterior?
      interacted_channels[ei] += filtered_noises[e]* CII* (CII**(elec_array.gs[e][ei]))
      #interacted_channels[ei] += filtered_noises[e]*elec_array.gs[e][ei]*CHANNEL_INTERACTION_INDEX (VIEJO)
  return interacted_channels

#Viejo. gain del crosstalk a "de" electrodos hacia la base.
#def interaction_basal_gain(de):
 # return 10.**(-de)

#Viejo gain del  crosstalk a "de" electrodos hacia el apex
#def interaction_apical_gain(de):
 # return 1/(10*(de**3)) #sería más correcto una exponencial como tenía antes, con una base algo menor a 10?

#def channel_interaction_weight(elec_array, )


# Elec Array Clase

In [ ]:
class Elec_Array:
  def __init__(self, N = 12, dx = 0, xi = 0, angularInsertion=0, type = MW, name = ''):
    self.Nchann = N
    self.dx =dx
    self.xi = xi
    self.type = type
    self.angularInsertion = angularInsertion
    self.freqsAnalisis = []
    self.freqsElectrodos = []
    self.channel1_lowcut = None
    self.preprocessing = None
    self.T_SPL = None
    self.C_SPL = None
    self.pw = None #Pulse width de cada canal en segundos
    self.SR = None
    self.name = name
    self.manufacturer = ''
    self.calcularFreqs()
    #self.gs = self.channel_interaction_matrix()
    self.input_sig = None
    self.output_sig = None
    #self.mismatch = mismatch

    #print('Creando array ', self.name)

    #todo meter en el proceso final un bool para el display 
    #Todo meter stft lado a lado antes y despues (tam con bool)
  
    #Por sanidad del código, mantener las frecuencias siempre ordenadas de menor a mayor (orden AB, al revés de Cochlear)

  def calcularFreqs(self):
    #print('Calculando frecuencias de elecs de ', self.name)
    if(self.dx > 0): 
      posiciones = np.array([self.xi -self.dx*i - MISMATCH for i in range(self.Nchann)]) #posicones x_ei de cada electrodo 
      #print('Posiciones: ', posiciones)
      if(self.type == LW):
        self.freqsElectrodos = Greenwood_p(posiciones/CDL_OC)
      if(self.type == MW):
        self.freqsElectrodos = Greenwood_Perimodiolar_MW_p(posiciones/CDL_MW)

    if(self.angularInsertion >0): #Si no, uso el dato de la inserción angular
      self.freqsElectrodos =   []
      for i in range(0,self.Nchann): self.freqsElectrodos.append(Greenwood_ang(self.angularInsertion*(self.Nchann-i)/self.Nchann))

  def setearParametros(self):
    if(self.manufacturer == COCHLEAR): self.SR = 16000
    if(self.manufacturer == MEDEL): self.SR = 17000
    if(self.manufacturer == AB): self.SR = 17400
    global IMPLANT_SR
    IMPLANT_SR = self.SR
    self.calcularFreqs() #setea Mismatch
    #self.gs = self.channel_interaction_matrix()

    global WIN_LEN_ACE
    if(MODE == INTERLEAVED and WIN_LEN_AUTO): WIN_LEN_ACE = round(PW*IMPLANT_SR) #analiza en c ciclo
    if(MODE == ACE and WIN_LEN_AUTO and self.manufacturer == COCHLEAR): WIN_LEN_ACE = 8 #ms
    if(MODE == ACE and WIN_LEN_AUTO and self.manufacturer == MEDEL): WIN_LEN_ACE = 4 #ms (chequear, no segurp)

    #Asigno fc de freqs de analisis
    if(type(self.freqsAnalisis[0]) == type([1,2])):
      for i in range(len(self.freqsAnalisis)):
        f1, f2 = self.freqsAnalisis[i]
        self.freqsAnalisis[i] = (f1+f2)/2
    return

  def channel_interaction_matrix(self):
    N= self.Nchann
    gs = np.zeros([N, N]) #Matriz de interacción de electrodos. Asumo que vienen de grave a agudo.
                          #gs[from][to] guarda el factor de escalamiento para lo que recibe el electrodo "to" al estimular el electrodo "from"
                          #Es una matriz cuadrada. Si la tomo como simétrica (la dispersión de corriente a "izq y der" es la misma, entonces su inversa es una matriz Tridiagonal)
                          #En el caso 
    l = 3.6 #mm, assumed space constant (patente CSSS Medel)

    for e_from in range(N):
      for e_to in range(N):
        if(e_to == e_from): g = 1

        d = abs(e_from-e_to)*self.dx
        if(self.freqsElectrodos[e_to] > self.freqsElectrodos[e_from]): #e_to es más basal que e_from
          #g = interaction_basal_gain(np.abs(ei-e)) * CHANNEL_INTERACTION_INDEX
          g = np.e**(-d/l)

        if(self.freqsElectrodos[e_to] < self.freqsElectrodos[e_from]): #e_to es más apical que e_from
         # g = interaction_apical_gain(np.abs(ei-e)) * CHANNEL_INTERACTION_INDEX
           g = np.e** ( (-d/l) * (1-ASYMMETRY_INDEX))
        gs[e_from][e_to] = g
    return gs
    #indice de asimetria = 0: Ambos se estimulan igual
    #indice de asimetria = 1: Hacia el apical decae mucho más lento (plano, casi conste). Hay más interacción.

    #Podría ajustar el ancho de banda de los filtros que se aplican sobre el ruido blanco como alternativa para simular esto, pero no tendría la asimetría
    #REvisar, no está bien que sean exponenciales para el modelo psicoacústico


## Calculo de frecuencias

In [37]:

#Devuelve la frecuencia característica de la distancia lineal  "x" a lo largo de la membrana basilar, medida en mm desde la ventana oval, a lo largo del órgano de Corti en el ducto coclear.
# Se asume un largo de de organo de corti de CDL_OC
def Greenwood(x, CDL_OC = 35):
    
    A = 165.4
    a = 2.1
    k = 0.88
    return A* ( 10 ** (a*(1-x/CDL_OC)) - k)
 
#Greenwood donde x está normalizado entre 0 y 1
def Greenwood_p(x):
    
    A = 165.4
    a = 2.1
    k = 0.88
    return A* ( 10 ** (a*(1-x)) - k)

#Mide 40cm, yo pongo 4cm -> internamente traduce al 10%

#Recibe una proporcion de distancia recorrida de OC (0 a 100 en %) y devuelve una proporcion de distancia recorrida en SG
#Va de (0,0) a (100,100)
# where y= percentage distance from base of the SG and x = percentage distance from base of the OC

def OC_to_SG_p(p, metodo = 2):
 
  if(metodo == 1):
    #Version 1. No usada (Divya Sridhar, Olga Stakhovskaya 2005)
    return -5.7*(10**-5) * p**3 + 0.0014*p**2 + 1.43*p   
 

  #Version 2: OLGA STAKHOVSKAYA 2006
  A = 0.22
  B = -0.93

  den = A*100/p + B*p/100 - A - B 
  return 100/(1+den**2)

#Igual pero a Modiolar Wall
def OC_to_MW_p(p):

  #OLGA STAKHOVSKAYA 2006

  A = 0.23
  B = -0.99

  den = A*100/p + B*p/100 - A - B 
  return 100/(1+den**2)

#Le das el angulo de rotacion y devuelve el porcentaje de OC recorrido de 0 a 100 (%)
def ang_to_OC_p(ang):
  A , B , C = -110, 0.002, 115
  return A*np.exp(-B*ang) + C

#Le das el angulo de rotacion y devuelve el porcentaje de SG recorrido (0 a 100)
def ang_to_SG_p(ang):
  A , B , C = -99.3, 0.004, 105
  return A*np.exp(-B*ang) + C

#Le das una proporcion del ganglio espiral y te devuelve el angulo al que corresponde (inversa de la de arriba)
def SG_p_to_ang(p):
  A , B , C = -99.3, 0.004, 105
  return -1/B * np.log((p-C)/A)


#Recibe una proporcion de recorrido de ganglio espiral y devuelve la proporcion correspondiente (matcheada en frecuencia) en el OC
def SG_p_to_OC_p(p, metodo = 1):

  if(metodo == 1):
    from pynverse import inversefunc
    return inversefunc(OC_to_SG_p, y_values=p, domain = (0,100), open_domain= (True, False), image=(0,100))
  if(metodo == 2):
    return ang_to_OC_p(SG_p_to_ang(p))

#Recibe una proporcion de recorrido de ganglio espiral en % y devuelve la proporcion correspondiente (matcheada en frecuencia) en el OC en %
def MW_p_to_OC_p(p):
    from pynverse import inversefunc
    return inversefunc(OC_to_MW_p, y_values=p, domain = (0,100), open_domain= (True, False), image=(0,100))


#Funcion de Greenwood adaptada para obtener la frecuencia de un punto a proporcion de inserción p (entre 0 y 1) a lo largo del ganglio espiral
def Greenwood_Perimodiolar_SG_p(p, metodo = 1):
    return Greenwood_p(SG_p_to_OC_p(p*100, metodo)/100)


#Funcion de Greenwood adaptada para obtener la frecuencia de un punto a proporcion de inserción p (entre 0 y 1) a lo largo de la pared modiolar
def Greenwood_Perimodiolar_MW_p(p):
    return Greenwood_p(MW_p_to_OC_p(p*100)/100)
#*100 y /100 es para normalizar los porcentajes



#Viejo: Devuelve la frecuencia característica de la posición "x" a lo largo de la membrana basilar, medida en mm desde la base
#asume longitud promedio de 35mm y rangos de audición promedios (G(0) y G(35)). Estudios recientes apuntan a largo promedio de 34.5mm (Danielan et al 2019) pero como es variable dejo este valor estándar
def Greenwood_estandar(x):
  return 165.4* ( 10 ** (0.06*(35-x)) - 0.88)

#Devuelve la frecuencia característica del ángulo "a" a lo largo de la membrana basilar, medido en grados desde la base
#Uso largo angular promedio 880°. Promediado de (Danielan et al 2019 y Anandhan 2018)
#--> Ángulo entre 0° y 880°
#No usar, no me está dando bien con los datos de (Anandhan ).
#No da buenos resultados porque esto asume una curva circular, donde el ángulo barrido es proporcional a la distancia recorrida, y en la cóclea no lo es
def Greenwood_ang(a):
  x = a/880*35
  return Greenwood(x)


# CIS - ACE - FSP

In [ ]:
##############################################################################################
#                             CIS,  ACE,  FSP                                               ##
###############################################################################################


def CIS_Sintesis(envs, elec_array):

  # 1- Asigno bandas de análisis a posiciones en la cóclea
  Nchann = elec_array.Nchann
  n = round(len(envs[0])*OUT_SR/IMPLANT_SR)

  #Resampleo las envolventes a la frecuencia de las señales filtradas
  #Resampleo s a algo que permita representar todo el rango audible. Elijo 48khz por ser multiplo entero de 16khzre
  for e in range(Nchann): envs[e] = resample(envs[e], IMPLANT_SR, OUT_SR)
  #TODO:  ABAJO SE TIENEN QUE RESAMPLEAR AL PULSE RATE EN REALIDAD!!!

  #Paso ruido blanco por los mismos filtros pasabanda y que las envolventes modulen su amplitud
  wnoise = wnoise_sig(n)
  filtered_noises = filter_bank_sintesis2(wnoise, elec_array)

  # 2- Simulo la interacción de canales (ya sobre el "medio físico")
  # Podría ajustar el ancho de banda de los filtros que se aplican sobre el ruido blanco como alternativa para simular esto, pero no tendría la asimetría
  #Esto es valid para modo Continous, pero no para interleaved


  # 3 - Mezclo los canales paralelos a uno solo
  if(MODE == CONTINOUS):  #Ruido blanco modulado
    out = np.zeros(n)
    filtered_noises = np.multiply(filtered_noises,envs)
    interacted_channels = channel_interaction_CONTINOUS(filtered_noises, elec_array)
    for c in range(Nchann): out += interacted_channels[c]
    return out

  #Maxima (simulacion de ACE)
  #Igual a CONTINOUS cuando MAXIMA está seteado a 0
  #Ojo que tecnicamente, m = #electrodos = N_chann en el nombre n-m, y yo los nombro al revés
  # m = cantidad de canales a estimular
  if(MAXIMA == 0): m = Nchann
  if(MAXIMA >  0): m = MAXIMA


  #Modo ACE continuo: Simplemente CIS continuo con menos canales
  if(MODE == ACE):
    return simulate_CIS_ACE2(envs, filtered_noises, elec_array, m)

  #Modo Interleaved: Simulo la estimulación secuencial (Y aplica  n of m = ACE)
  #TODO: Probar la alternativa de implementar CONTINOUS, y hacer 0 los otros canales de a momentos.
  if(MODE == INTERLEAVED):
     return simulate_CIS_Interleaved(envs, filtered_noises, elec_array, m)

  return

'''
Nota: Así como están ahora estos modos a partir de v3, son más fieles que en la versión 2
Un buen experimento a hacer sería comparar el cambio entre las dos versiones (basicamente, cambia el modo de aplicar channel interaction)
entre esta manera más fiel y la otra más naive, a ver si justifica la fidelidad contra tiempo de ejecucion.
Nota v5: Revisar y comparar con la última versión, este y Interleaved
'''
def simulate_CIS_ACE(envs, filtered_noises, elec_array, m):
    n = len(filtered_noises[0]) #nSamps
    out = np.zeros(n)
    gs = elec_array.gs

    WIN_SIZE = round(WIN_LEN_ACE * OUT_SR/1000) #en sampless

    ''' 
    envAce = np.zeros(WIN_SIZE) #Pequeña envolvente que se le aplica a cada estimulo
    #Raising sine window
    for i in range(WIN_SIZE): 
      if(i<WIN_SIZE//8):
        envAce[i] = sin(2*pi*i*2/WIN_SIZE)

      elif i< WIN_SIZE*7//8: envAce[i] = 1.
    
      else: envAce[i] = - sin(2*pi*i*2/WIN_SIZE)
    signo = 1
    '''

    envACE = Tukey(WIN_SIZE, 0.25) #para encender los nuevos
    envOn = envACE.copy()
    envOn[WIN_SIZE//2: WIN_SIZE] = np.ones(WIN_SIZE//2)

    envOff = envACE.copy()
    envOff[0:WIN_SIZE//2] = np.ones(WIN_SIZE//2)


    i = 0
    power = 2 #Norma con la que extraigo la envolvente: 1 o 2. 
    encendidosAnterior = [] #canales seleccionados en el ciclo anterior. No se les aplica ventana.
    while(i+WIN_SIZE<n):

      #Extraigo las envolventes de mayor amplitud
      amplitudes_promedio = [sum(np.array(envs[c][i: i+WIN_SIZE]**power)) for c in range(elec_array.Nchann)] #Tomo la amplitud promedio en el ciclo de estimulación de cada envolvente
      indices_max = np.argpartition(amplitudes_promedio, -m)[-m:] #Indices de las envolventes de mayor amplitud

      #Estimulo bandas maximas
      for max_channel in indices_max:
        
        #prender apagar, no es necesario aun
        #if(max_channel in encendidosAnterior): 
                                                                          # envs[max_channel][i: i+WIN_SIZE] Trackeo la envolvente# todo el ciclo
        #out[i: i+WIN_SIZE] += filtered_noises[max_channel][i: i+WIN_SIZE] * amplitudes_promedio[max_channel] #* envAce *signo # misma amp durante todo el pulso
        out[i: i+WIN_SIZE] += filtered_noises[max_channel][i: i+WIN_SIZE] * envs[max_channel][i: i+WIN_SIZE] 

        CII = CHANNEL_INTERACTION_INDEX
        #Simulo channel interaction:
        if(CII > 0):
          for other_channel in range(elec_array.Nchann):
            if(other_channel == max_channel): continue 
            out[i:i+WIN_SIZE] += filtered_noises[other_channel][i:i+WIN_SIZE] * CII * envs[max_channel][i:i+WIN_SIZE]*gs[max_channel][other_channel]
            #v2 out[i:i+WIN_SIZE] += filtered_noises[other_channel][i:i+WIN_SIZE] * amplitudes_promedio[max_channel]*gs[max_channel][other_channel]
            #v3 out[i:i+WIN_SIZE] +=  filtered_noises[other_channel][i:i+WIN_SIZE]* CII*(2**(envs[max_channel][i:i+WIN_SIZE]*elec_array.gs[max_channel][other_channel]*(1+CII)))
                                                                                  #envs o amplitudes promedio...   

      encendidosAnterior = indices_max.copy()
      i+=WIN_SIZE
      
    return out

#Vuelo muchas cosas que no sé exactamente como implementan los implantes
def simulate_CIS_ACE2(envs, filtered_noises, elec_array, m, separate = False):
    if(DEBUG): separate = True
    n = len(filtered_noises[0]) #nSamps
    out = np.zeros(n)
    if(separate):  outs = np.zeros((elec_array.Nchann, n))


    WIN_SIZE = round(WIN_LEN_ACE * OUT_SR/1000) #en sampless

    i = 0
    power = 1 #Norma con la que extraigo la envolvente: 1 o 2. 
    while(i+WIN_SIZE<n):

      #Extraigo las envolventes de mayor amplitud
      amplitudes_promedio = [sum(np.array(envs[c][i: i+WIN_SIZE]**power)) for c in range(elec_array.Nchann)] #Tomo la amplitud promedio en el ciclo de estimulación de cada envolvente
      indices_max = np.argpartition(amplitudes_promedio, -m)[-m:] #Indices de las envolventes de mayor amplitud

      #Estimulo bandas maximas
      for max_channel in indices_max:
            out[i: i+WIN_SIZE] += filtered_noises[max_channel][i: i+WIN_SIZE] * envs[max_channel][i: i+WIN_SIZE]                                                      #envs o amplitudes promedio...   
      
      if(separate):
        for max_channel in indices_max:
              outs[max_channel][i: i+WIN_SIZE] += filtered_noises[max_channel][i: i+WIN_SIZE] * envs[max_channel][i: i+WIN_SIZE]                                                      #envs o amplitudes promedio...   
      
      i+=WIN_SIZE
      
    if(separate): return outs
    else: return out


def simulate_CIS_Interleaved(envs, filtered_noises, elec_array, m):
  i = 0
  pwn = round(PW*OUT_SR) #samples en cada pulso
  n = len(filtered_noises[0])
  #pwn = round(1/m/PPS) #samples activos en cada pulso
  gs = elec_array.gs
  out = np.zeros(n)

  while(i+pwn<n):

    #Para acelerar: Si no hace falta buscar los máximos (n == m en n of m), no los busco
    if(m<elec_array.Nchann):
      #Extraigo las envolventes de mayor amplitud
      #amplitudes_promedio = [sum(envs[c][i:i+pwn]) for c in range(N_chann)] #Tomo la amplitud promedio en el ciclo de estimulación de cada envolvente
      amplitudes_promedio = [sum(np.array(envs[c][i:i+pwn]**2)) for c in range(elec_array.Nchann)]
      indices_max = np.argpartition(amplitudes_promedio, -m)[-m:] #Indices de las envolventes de mayor amplitud
    else: indices_max = range(elec_array.Nchann)

    for max_channel in indices_max:
      #if(len(interacted_channels_maximos[c][fr: to])<pwn): continue  #LLegué al final de la señal   (todo: filtered_noises?)

      fr = i+max_channel*pwn
      to = i+(max_channel+1)*pwn

      out[fr: to] = filtered_noises[max_channel][fr: to] * envs[max_channel][fr: to]

      #Simulo channel interaction:
      if(CHANNEL_INTERACTION_INDEX>0):
        for other_channel in range(elec_array.Nchann):
          if(other_channel == max_channel): continue
          out[fr:to] += filtered_noises[other_channel][fr:to] * envs[max_channel][fr:to]*gs[max_channel][other_channel] #VIEJOOOO


    i+=m*pwn

  return out



## Definimos CIS

(Cuando "MAXIMA < #canales", se convierte en ACE)

In [ ]:
class Elec_Array(Elec_Array):

  def CIS(self, s, input_sr = None):
    N = self.Nchann

    # 0a - Seteo parámetros del implante
    self.setearParametros()

    #0b - Resampleo a 16khz - Hecho al cargar el audio, si no se especifica (sr=0)
    if(input_sr!=0):
      self.input_sig = resample(s, input_sr, self.SR)
    else:
      self.input_sig = s.copy()

    # 1 - Pre-énfasis
    b, a = signal.butter(1, 1200,btype = 'highpass', fs = self.SR , output='ba')
    #zi = signal.lfilter_zi(b, a)
    #self.input_sig, _ = signal.lfilter(b, a, self.input_sig, zi=zi*self.input_sig[0])
    self.input_sig = signal.lfilter(b, a, self.input_sig) 

    #1.5 AGC
    #todo reimplementar solo la parte relevante

    #2 - Banco de filtros pasabanda de análisis
    filtered_bank = filter_bank_analisis(self.input_sig, self, ORDEN)

    #3 - Extracción de envolventes
    if(self.manufacturer == COCHLEAR): envelopes = [envelope_Cochlear(filtered_bank[i], ENVELOPE_FC, ENVELOPE_FULLWAVE) for i in range(N)] # CIS convencional
    if(self.manufacturer == MEDEL): envelopes = [envelope_Hilbert(filtered_bank[i]) for i in range(N)] # "CIS+"

    #4 - Compresión del rango dinámico
    #compressed_envelopes = [compress_envelope(envelopes[i]) for i in range(N)]

    #5 - Aplicamos CIS

    #Aquí ocurre:
    #5.1 Asignación de bandas a electrodos + Mismatch
    #5.2 - Channel Interaction
    out = CIS_Sintesis(envelopes, self)

    return out


## FSP

In [40]:
'''Recibe una señal y genera tramos de senoidales que se ajusten a los zero-crossing
A diferencia de lo que hace el implante, yo elegí tomar pares de zero crossings en vez de solo los que sean
rising de neg a pos.
¿Hace mucha diferencia? No lo sé. Lo elegí por un tema de pragmatismo a la hora de implementar

Observacion: Algo más sofisticado, sería aprovechar que sen(x) \approx x cuando x \approx 0,
 Y entonces estimar los valores de x,
Para tener yo un valor estimado del período mucho más fino a la hora de hacer la resíntesis
Simplemente porque el implante en teoría no lo hace, entonces yo tampoco
'''
def track_zero_crossings_sin(s, continous_phase = True):
  s0 = [1 if s[i] > 0 else -1 for i in range(len(s))] #señal de signos
  out = np.zeros(len(s0))
  i=0
  phi = 0 #fase
  if(s[0] <0): phi = pi
  while i <len(s0):
    i0 = i
    i+=1 #Importante: Este +1 tiene que estar al inicio, no al final del loop. Sino genera una distorsión (que puede ser interesante en otro contexto) al dejar samples seteados en 0 en cada ciclo

    #Busco primer zero-crossing
    while(i <len(s0) and
          s0[i]*s0[i-1]>0): i+=1
    if( i>= len(s0)): break
    i+=1

    #Busco segundo zero-crossing
    while(i <len(s0) and
          s0[i]*s0[i-1]>0): i+=1
    di = i-i0 #Período en samples (sin interpolar)

    if continous_phase:
      for j in range(i0, i): #Lleno un período
        out[j] = sin(phi)
        phi += 2*pi/di  #Seria lo mismo que el "else" con range(0,di) en vez de ese range

    else:
      for j in range(i0, i): out[j] = sin(2*pi*j/di) #Lleno un período

  return out
#Guardo el parámetro "Continous_phase" porque me pregunto si, el ruido que resulta al dejarlo en false,
#no es similar al ruido que escucha un implantado con este método. Queda a definir

#Recibe una señal y devuelve una lista del mismo largo con las frecuencias estimadas en cada instante a partir de los zero crossings
def track_zero_crossings_freqs(s):
  s0 = [1 if s[i] > 0 else -1 for i in range(len(s))] #señal de signos
  out_freqs = np.zeros(len(s0))
  i=1

  #Busco primer zero-crossing para empezar a partir de ahí
  while(i <len(s0) and
        s0[i]*s0[i-1]>0): i+=1
  if( i>= len(s0)): return out_freqs

  #El primer zero crossing está en s0[i-0.5]

  i0 = i
  f = 0
  fnew = 0
  #Busco los siguientes y estimo las frecuencia
  while i <len(s0):

    #cambio de signo, hubo zero-crossing
    if(s0[i] != s0[i0]):
      di = i-i0 #Período en samples (sin interpolar)
      fnew = IMPLANT_SR / di /2
      i0 = i

    f = .5* fnew +.5*f #pasabajos simple
    out_freqs[i] = f
    i+=1

  return out_freqs

#Recibe una señal y devuelve una lista del mismo largo con las frecuencias estimadas en cada instante a partir de los zero crossings
#Igual al de arriba pero solo trigerea en sentidos "positivos" de cruce
def track_zero_crossings_freqs2(s):
  s0 = [1 if s[i] > 0 else -1 for i in range(len(s))] #señal de signos
  out_freqs = np.zeros(len(s0))
  i=1

  #Busco primer zero-crossing para empezar a partir de ahí
  while(i <len(s0) and
        s0[i]*s0[i-1]>0): i+=1
  if( i>= len(s0)): return out_freqs

  #El primer zero crossing está en s0[i-0.5]

  i0 = i
  f = 0
  fnew = 0
  i +=1
  #Busco los siguientes y estimo las frecuencia
  while i <len(s0):

    #cambio de signo, hubo zero-crossing (sentido positivo)
    if(s0[i] != s0[i-1] and s0[i] == s0[i0]):
      di = i-i0 #Período en samples (sin interpolar)
      fnew = IMPLANT_SR / di
      i0 = i


    f = .5* fnew +.5*f #pasabajos simple
    out_freqs[i] = f
    i+=1

  #Para no tener problemas luego, las primeras freq que son 0 las igualo a las que no sean
  z = 0
  while(out_freqs[z]==0): z+=1
  while(z>0): out_freqs[z] = out_freqs[z+1]

  return out_freqs

def filter_bank_FSP(s, elec_array, freqs_apicales, orden = ORDEN):
  N = elec_array.Nchann
  fs = OUT_SR

  #Freqs_apicales tiene las frecuencias instantáneas de los dos canales más graves

  #Out (array con todas las señales filtradas)
  filtered_apicales= np.zeros(len(freqs_apicales), len(s))

  #Todo: Que el primer low cut sea mucho más pronunciado (más orden)
  #e1 --> N-1
  for chann in len(freqs_apicales):
    i = 0
    freqs = freqs_apicales[chann]
    filtered = filtered_apicales[chan]
    while i < len(freqs_apicales):
      i0 = i
      while(freqs[i+i] != freqs[i]): i+=1

      c = DISPERTION/2
      eps = 0.5 #Por si DISPERTION == 0
      fc = freqs[i]
      f1 = fc**(1-c) - eps
      f2 = fc**(1+c) + eps

      stramito = s[i0:i0]


      filtered_tramito = aplicar_filtro(stramito, orden, fs, f1, f2)
      filtered[i0:i] = filtered_tramito

  #eN --> Nyquist

  return filtered_apicales

def simulate_FSP(envelopes, elec_array, filtered_apicales, NApic = 2):
  from copy import copy
  #atajo provisorio, luego enmendar: "Desactivo" a la fuerza los canales graves
  envelopes_copy = envelopes[NApic:]
  elec_array_copy = copy(elec_array)
  elec_array_copy.freqsAnalisis = elec_array_copy.freqsAnalisis[NApic:]
  out = simulate_CIS(envelopes_copy, elec_array_copy)

  for e in range(NApic): envelopes[e] = resample(envelopes[e], IMPLANT_SR, OUT_SR)

  #Paso ruido blanco por los mismos filtros pasabanda y que las envolventes modulen su amplitud
  wnoise = wnoise_sig(len(envelopes[0]))
  freqs_apicales = [track_zero_crossings_freqs2(filtered) for filtered in filtered_apicales]
  filtered_noises_apic = filter_bank_FSP(wnoise, elec_array, ORDEN, freqs_apicales)

  out += filtered_noises_apic

  # 3 - Mezclo los canales paralelos a uno solo
  interacted_channels = channel_interaction_CONTINOUS(filtered_noises, elec_array)
  for c in range(Nchann): out += interacted_channels[c] * np.array(envs[c])
  return out


In [41]:
def FSP(s, elec_array):
  N = elec_array.Nchann

  # 0b - Seteo parámetros del implante
  elec_array.setearParametros()

  # 1 - Pre-énfasis
  b, a = signal.butter(1, 1200,btype = 'highpass', fs = IMPLANT_SR , output='ba')
  zi = signal.lfilter_zi(b, a)
  s, _ = signal.lfilter(b, a, s, zi=zi*s[0])

  #2 - Banco de filtros pasabanda de análisis
  filtered_bank = filter_bank_CIS(s, elec_array, ORDEN, analisis = True)

  #3 - Extracción de envolventes
  envelopes = [envelope_Hilbert(filtered_bank[i]) for i in range(N)] # "CIS+"

  #4 - Compresión del rango dinámico
  compressed_envelopes = [compress_envelope(envelopes[i]) for i in range(N)]

  #5 - Aplicamos CIS

  #Aquí ocurre:
  #5.1 Asignación de bandas a electrodos + Mismatch
  #5.2 - Channel Interaction
  NApic = 2
  out = simulate_FSP(envelopes, elec_array, filtered_bank[:NApic], NApic)

  return out
#Entre 70 y 350Hz
#Elijo este método: En los canales con CSSS (Elijo los primeros 3, según datos de calibración de Francisco) sumo en paralelo la señal "sinusoidal" que daría el método,
#al ruido original
#O sea, replico el método CSSS a mano en el paso de resíntesis
#Esto se logra disparando un pulso en cada zero crossing.
#De esta forma, simulo la parte espacial (electrodo y ruido) y temporal fina (csss y zero-crossings)


#La razón por la que elijo implementar el método de zero-crossings igual, a pesar de que para tonos simples podría resolverlo
#sumando la señal filtrada en paralelo, es que cuando hay más de un tono que cae en la misma banda,
#los zero crossing no dan un patrón de repetición acorde a una frecuencia fundamental "buena"


## Funciones gráficas

In [ ]:
def plot_stft_señalsola(x, lineas = []):
  x = np.array(x)
  n_fft = 4096
  win_length = n_fft
  hop_length = n_fft//4

  #Espectro
  x_stft = np.abs(librosa.stft(x, n_fft=n_fft, win_length = win_length, hop_length = hop_length)) #hop = n_fft//4
  x_stft_db = librosa.amplitude_to_db(x_stft, ref=np.max, top_db = 60)
  n_frames = len(x_stft_db[0])

  #Grafico
  fig, ax = plt.subplots()
  ax.set_title('Espectrograma')
  fig.set_size_inches(7, 3.5)
  ax.set_ylim(0,IMPLANT_SR/2)

  img = librosa.display.specshow(data=x_stft_db, y_axis='log', x_axis = 'time',ax=ax,
                            sr=IMPLANT_SR, cmap= 'magma',
                            n_fft=n_fft, win_length = win_length, hop_length = hop_length)

  plt.colorbar(img, ax= ax, format='%+2.0f dB')
  plt.hlines(lineas, 0.1, len(x)/IMPLANT_SR*0.9 , colors='yellow', linestyle = 'dotted')

  plt.show()
  return fig, ax

#Analisis es True para señales de entrada al implante, False para señales de salida del simulador
def plot_stft(x, elec_array, analisis=False, show_black_and_white=False, mini = False, top_db = 40, mostrarLineas = True):

  x = np.array(x)
  n_fft = 512
  win_length = n_fft
  hop_length = n_fft//4

  #Espectro
  x_stft = np.abs(librosa.stft(x, n_fft=n_fft, win_length = win_length, hop_length = hop_length)) #hop = n_fft//4
  x_stft_db = librosa.amplitude_to_db(x_stft, ref=np.max, top_db = top_db)
  n_frames = len(x_stft_db[0])

  #Grafico
  fig, ax = plt.subplots()
  if(mini): fig.set_size_inches(7, 3.5)
  else: fig.set_size_inches(15, 7)
  #ax.set_ylim([0, 1])

  #Seteo samplig rate según si es señal de entrada al implante o salida del simulador
  if(analisis): sampling_rate = IMPLANT_SR
  else: sampling_rate = OUT_SR


  if(show_black_and_white): cmap = 'gray_r'
  else: cmap = 'magma'
  img = librosa.display.specshow(data=x_stft_db, y_axis='log', x_axis = 'time',ax=ax,
                            sr=sampling_rate, cmap= cmap,
                            n_fft=n_fft, win_length = win_length, hop_length = hop_length)

  plt.colorbar(img, ax= ax, format='%+2.0f dB')


  #INPUT SIGNAL:
  if(mostrarLineas and analisis): #Mostramos las frecuencias de analisis
    ax.set_title('Espectrograma. Azul = Freqs Analisis')
    plt.hlines(elec_array.freqsAnalisis, 0.1, len(x)/sampling_rate*0.9 , colors='blue')
    #for f in elec_array.freqsAnalisis:  #Agregamos lineas horizontales en las frecuencias centrales de los electrodos del array
      #lineaX = [512*i/OUT_SR for i in range(n_frames)] #fftsize//4
      #lineaY = [f for _ in range(n_frames)]
      #plt.plot(lineaX, lineaY, '-b')
      #ylim(0,8000)

 #OUTPUT SIGNAL:
  if(mostrarLineas and not analisis): #Mostramos las frecuencias de los electrodos
      ax.set_title('Espectrograma. Rojo = Freqs Electrodos')
      plt.hlines(elec_array.freqsElectrodos, 0.1, len(x)/sampling_rate*0.9 , colors='red')
      #plt.ylim(0,18000)

  ax.set_ylim(0,min(14000, sampling_rate/2))

  '''
    for f in elec_array.freqsElectrodos: #Agregamos lineas horizontales en las frecuencias centrales de los electrodos del array
      lineaX = [512*i/OUT_SR for i in range(n_frames)] #fftsize//4
      lineaY = [f for _ in range(n_frames)]
      plt.plot(lineaX, lineaY, '-r')
      plt.ylim(0,18000)
  '''

  plt.show()
  return 
  #return fig, ax


def plot_fft(s, elec_array, f_from = 20, f_to = 0, logx = True, range_dB = 40, norm = 1., showElecs = False, show = True, sr = OUT_SR, step = 2):

  from scipy.fft import rfft, rfftfreq
  
  T = 1/sr #periodo
  N = len(s)
  t = np.arange(0, N*T, T) # Sampling times

  # FFT and bin centers
  y = rfft(s)
  y = np.abs(y)*2/N
  y /= norm
  #y = 20*np.log10(np.abs(y)) #dB
  y_db = 20*np.log10(y)
  f = rfftfreq(N, T)

  #df : sr/N  --> f(i) = i* sr/N --> i = f*N/sr  
  if(f_to == 0): fto = sr/2
  #ifrom = round(f_from*N/sr) #ah.... cuando no sabia del ax.set_xlim...
  #ito = round(f_to*N/sr)

  # Plot spectrum
  fig, ax = plt.subplots(dpi = 150) #default dpi is 100
  if(logx): ax.set_xscale('log')
  ax.set_xlim(f_from, f_to)

  #ax.plot(f[ifrom:ito], y[ifrom:ito])
  #ax.semilogy(f[ifrom:ito], y[ifrom:ito], color = 'black', linewidth = 0.5) eje logaritmico, y lineal
  #ax.set_ylim(10**(-range_dB/20)*max(y), max(y)*1.1)

  #ax.plot(f[ifrom:ito], y_db[ifrom:ito], color = 'black', linewidth = 0.5) #eje lineal, y logaritmico
  ax.plot(f, y_db, color = 'black', linewidth = 0.5) #eje lineal, y logaritmico
  
  y_max =  max(y_db) +5
  y_min = max(y_db) -range_dB
  ax.set_ylim(y_min , y_max)

  plt.xlabel('Frecuencia (Hz)')
  plt.ylabel('Amplitud (dB)')  
  plt.grid(alpha  = 0.8)

  #AGREGAMOS MARQUITAS PARA LOS ELECTRODOS
  if(showElecs):
    ''' VIEJOO
    #Consigo las marquitas originales
    x_ticks = ax.get_xticks()
    x_labels = ax.get_xmajorticklabels()
    ax.set_xticks(list(x_ticks) + elec_array.freqsElectrodos[::2], labels = list(x_labels) + elec_labels[::2])
    '''

    color = 'limegreen'
    secax = ax.secondary_xaxis('top') #eje secundario arriba
    secax.set_xlabel('(Frecuencia característica de los electrodos)', color = 'dimgray', style = 'italic')
        

    #Creo marquitas para los elec
    elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]

    #Agrego marquitas
    step = step
    secax.set_xticks(elec_array.freqsElectrodos[::step], labels = elec_labels[::step])
    
    #Les pongo color
    for i in range(1,elec_array.Nchann//step+1):
      secax.get_xticklabels()[-i].set_color(color)
      secax.get_xticklabels()[-i].set_rotation(45)
      secax.get_xticklabels()[-i].set_rotation_mode('anchor')


    ax.vlines(elec_array.freqsElectrodos,y_min , y_max , colors=color, linewidth = 1.3, linestyle = (0, (1,5)), alpha = 0.65)

    """ #lineas verticales con los electrodos
    ## get the xticks, which are the numeric location of the ticks
    xticks = ax.get_xticks()

    # get the xticks and convert the values in the array to str type
    xticklabels = list(map(str, ax.get_xticks()))

    # update the string to be changed
    xticklabels[1] = 'Test'

    # set the xticks and the labels
    _ = ax.set_xticks(xticks, xticklabels) """
  
  if(show): plt.show()

  return fig, ax

def logear_experimento(nombre, s, elec_array):
  return
  #Exportar txt con nombre del experimento y parámetros, audio de salida, etc


# Modelos de Arrays

In [43]:
#MEDEL---------------------------------------------------------------------------
freqTableSonnet = [[100,198],[198, 325], [325,491], [491, 710], [710, 999], [999, 1383], [1383, 1893], [1893, 2574], [2574, 3483],  [3483, 4698], [4698, 6323], [6323, 8500]] #Mapa de los SONNETS. Paper Wolfe/NOgueira
freqTableSonnetb = [[100,198],[198, 325], [326,492], [493, 712], [707, 995], [992, 1373], [1378, 1886], [1888, 2567], [2600, 3528], [3466, 4703], [4799, 6512], [6239, 8466]] #Nogueira (y wolfe)
freqs_estandar = [149,262,409,602,851, 1183,1632,2228,3064,4085,5656,7352] #Implante Sonnet Logarithmic FS 100-8500. Wolfe, capitulo de Programming Medel. parecen los puntos medios de las de arriba
#frecuencias centrales de TableSonnet: [149, 262, 408, 600, 854, 1191, 1638, 2234, 3028, 4090, 5510, 7412]

GENERIC_MEDEL1 = Elec_Array(N=12, xi = 31.5, dx=26.4/11, type = LW) #monopolar  Jace Wolfe p339
GENERIC_MEDEL1.freqsAnalisis = freqs_estandar
GENERIC_MEDEL1.name = 'FLEXSOFT/FLEX31'
GENERIC_MEDEL1.manufacturer = MEDEL

GENERIC_MEDEL2 = Elec_Array(N=12, xi = 30.87, dx = 2.18, type = LW)
GENERIC_MEDEL2.freqsAnalisis = freqs_estandar
GENERIC_MEDEL2.name = 'Generic Medel 2 Sonnet'
GENERIC_MEDEL2.manufacturer = MEDEL
GENERIC_MEDEL2.channel1_lowcut = 100

GENERIC_MEDEL2b = Elec_Array(N=12, xi = 30.87, dx = 2.18, type = LW)
GENERIC_MEDEL2b.freqsAnalisis = freqTableSonnetb
GENERIC_MEDEL2b.name = 'Generic Medel 2 Sonnet'
GENERIC_MEDEL2b.manufacturer = MEDEL


#El de Francisco, Flex28
GENERIC_MEDEL3 = Elec_Array(N=12, xi = 28, dx = 23.1/11, type = LW) 
GENERIC_MEDEL3.freqsAnalisis = [120,235,384,579,836, 1175,1624,2222,3019,4084,5507,7410] #Logarithmic FS 70-8500
GENERIC_MEDEL3.name = 'Medel Francisco Opus 2 FLEX28'
GENERIC_MEDEL3.manufacturer = MEDEL
GENERIC_MEDEL3.channel1_lowcut = 70 #FSP = 70Hz, HDCIS = 250Hz


GENERIC_MEDEL4 = Elec_Array(N=12, xi = 28, dx = 23.1/11, type = LW) #El de la patente de CSSS
GENERIC_MEDEL4.freqsAnalisis = [393,494,622,782,983, 1237,1556,1958,2463,3098,3898,4936] #Y llega hasta 5500, no Nyquist
GENERIC_MEDEL4.name = 'Medel Patente'
GENERIC_MEDEL4.manufacturer = MEDEL
GENERIC_MEDEL4.channel1_lowcut = 350 #FSP = 70Hz, HDCIS = 250Hz

GENERIC_MEDEL5 = Elec_Array(N=12, xi = 24, dx=20.9/11, type = LW) #Jace Wolfe p339
GENERIC_MEDEL5.freqsAnalisis = freqs_estandar
GENERIC_MEDEL5.name = 'FLEX24'
GENERIC_MEDEL5.manufacturer = MEDEL

GENERIC_MEDEL6 = Elec_Array(N=12, xi = 31.5, dx=26.4/11, type = LW) #Jace Wolfe p339 igual al 1 salvo el nombre
GENERIC_MEDEL6.freqsAnalisis = freqs_estandar
GENERIC_MEDEL6.name = 'Standard'
GENERIC_MEDEL6.manufacturer = MEDEL


'''
#AB---------------------------------------------------------------------------
GENERIC_AB1 = Elec_Array(N=16)
GENERIC_AB1.freqsAnalisis = [333, 455, 540, 642, 762, 906, 1076, 1278, 1518, 1803, 2142, 2544, 3022, 3590, 4264, 6645]  #fuente: Wolfe , capitulo Programming AB
GENERIC_AB1.channel1_lowcut = 250

GENERIC_AB2 = Elec_Array(N=16) #Igual excepto la primer banda
GENERIC_AB2.freqsAnalisis = [383, 455, 540, 642, 762, 906, 1076, 1278, 1518, 1803, 2142, 2544, 3022, 3590, 4264, 6645]  #fuente: Wolfe , capitulo Programming AB
GENERIC_AB1.channel1_lowcut = 350
'''

#Cochlear ---------------------------------------------------------------------------
#NUCLEUS_22 = Elec_Array(N=22, dx = 0.75)
freqTable22 = [[188,313], [313,438], [438, 563], [563, 688], [688, 813], [813,938], [938, 1063], [1063, 1188], [1188, 1313], [1313, 1563], [1563, 1813], [1813, 2063], [2063, 2313], [2313, 2688], [2688, 3063], [3063, 3563], [3563, 4063], [4063, 4688], [4688, 5313], [5313, 6063], [6063, 6938], [6938, 7938]  ] #fuente: Wolfe , capitulo Programming  Nucleus
#                  1         2          3           4           5           6           7               8           9               10          11          12              13          14              15              16          17              18          19              20          21              22          
#freqTable22 = freqTable22[::-1]
#electrodes that are 1.5mm apart are used as bipolar pairs



#AXEL OIDO DERECHO
Cochlear_Axel_Der = Elec_Array(N=22, xi= 20, dx = 0.75, type = LW, name = 'Axel Derecho CI24M - Nucleus 24 199x') #xi puede estar mal
Cochlear_Axel_Der.freqsAnalisis = freqTable22
Cochlear_Axel_Der.manufacturer = COCHLEAR
# electrode surface area is largest for basal electrodes (0.565 mm2) andsmallest for apical electrodes (0.377 mm2; van den Honert, personal communication).

#AXEL OIDO IZQUIERDO
Cochlear_Axel_Izq = Elec_Array(N=22, xi= 18.7, dx = 14.25/21, type = MW, name = 'Axel Izquierdo Implante Freedom (CI24RE) Contour Advance 201x') 
Cochlear_Axel_Izq.freqsAnalisis = freqTable22
Cochlear_Axel_Izq.manufacturer = COCHLEAR
#OJO QUE dx NO ES CTE TECNICAMENTE ACA
#TODO implementar ese dx variable (se puede medir viendo el manual)


#Prototipo como el Implante Freedom (CI24RE) + Contour Advance + N6 de Axel
#OJO: El Contour Advance tiene dx variable, de 0.4 a 0.8mm, y cubre un area de estimulacion de 14.25mm. Esto sale del manual
#NO USAR, Es perimodiolar, no Lateral Wall
#GENERIC_COCHLEAR1 = Elec_Array(N=22, xi= 19, dx = 15/21, type = MW) #Tipo el de Axel
#GENERIC_COCHLEAR1.freqsAnalisis = freqTable22
#GENERIC_COCHLEAR1.name = 'Generic Cochlear 1'
#GENERIC_COCHLEAR1.manufacturer = COCHLEAR
# Se corresponde a la Frequency Table #22, la tabla por defecto para Cochlear
#The electrode contacts in the Slim Modiolar Electrode (y Countour Advance también) 
# array are spaced in a nonlinear manner to maximize the uniformity of spectral coverage for a given number of electrodes and to promote a higher probability of good speech perception.
#  It should be noted that less than 1 mm typically exists between electrode contacts (Wolfe)

GENERIC_COCHLEAR1b = Elec_Array(N=19, xi = 19, dx =15/21)
GENERIC_COCHLEAR1b.freqsAnalisis = [[188,313], [313,438], [438, 563], [563, 688], [688, 813], [813,938], [938, 1063], [1063, 1313], [1313, 1563], [1563, 1813], [1813, 2188], [2188, 2563], [2563, 3063], [3063, 3563], [3563, 4188], [4188, 4938], [4938, 5813], [5813, 6813], [6813, 7938]  ] #fuente: Wolfe , capitulo Programming  Nucleus
#GENERIC_COCHLEAR1.freqs = [250, 375, 500, 625, 750, 875, 1000, 1188, 1438, ... ] frecuencias centrales
# Se corresponde a la Frequency Table #22 con menos electrodos activos

#Prototipo con el array Slim Straight (no usar aun)
GENERIC_COCHLEAR2 = Elec_Array(N=19, angularInsertion = 435)
GENERIC_COCHLEAR2.freqsAnalisis = freqTable22

#Converting the cochlear duct lengty (lateral wall) to cochlear duct lenght (organ of corti) can now be simply done by multiplying CDLLW by a factor of 0.87 z 0.9.

#Slim Straight
GENERIC_COCHLEAR3 = Elec_Array(N=22, xi= 24, dx = 19/21, type = LW ) #xi es 25mm pero el primer electrodo esta a 24mm
GENERIC_COCHLEAR3.freqsAnalisis = freqTable22
GENERIC_COCHLEAR3.name = 'Slim Straight'
GENERIC_COCHLEAR3.manufacturer = COCHLEAR

#Slim Straight - insertado a otra profundidad
GENERIC_COCHLEAR3b = Elec_Array(N=22, xi= 20, dx = 20/21, type = LW )
GENERIC_COCHLEAR3b.freqsAnalisis = freqTable22
GENERIC_COCHLEAR3b.name = 'Slim Straight (xi = 20mm)'
GENERIC_COCHLEAR3b.manufacturer = COCHLEAR

#TODO: En vez de greenwood distancia lineal, usar greenwod fraccional y un parámetro extra para "cochlear duct length (OC)". O directamente angular
'''
slim_straight = { N = 22, dx = 19.1/21, xi = None, type = LW}
slim_modiolar = { N = 22, dx = 14.4/21, xi = None, type = MW}
nucleushybridL24 = { N = 22, dx = 15/21, xi = None}
slim20 = { N = 22, dx = 19.1/21, xi = 20}
'''

#Datos de los manuales de implantes (en la carpeta Manuales/)

'\nslim_straight = { N = 22, dx = 19.1/21, xi = None, type = LW}\nslim_modiolar = { N = 22, dx = 14.4/21, xi = None, type = MW}\nnucleushybridL24 = { N = 22, dx = 15/21, xi = None}\nslim20 = { N = 22, dx = 19.1/21, xi = 20}\n'

Ahora ya está, pero para rehacer bien, separar las clases "Procesador" de "Implante Interno / Elec Array"

Poner en tesis: Electrodos estimados a intervalos de 180-200Cents (lateral) para este array de cochlear, y 500-700 para Medel

Ponerlo en la seccion 3. Haciendo referencia al ultimo grafico